In [6]:
# =========== IMPORT LIBRARIES ===========
from utils.utils import *
import pickle
import pprint

In [7]:
# =========== IMPORT DATASETS ===========
train = pd.read_csv("data/train_data_ner.csv")

# =========== NAN VALUES ===========
train['words'] = train['words'].astype(str)

# =========== CONVERT DATASETS ===========
X_train, y_train = transform_data_sentence_tag(train)

# =========== CREATE DICTIONARIES WITH UNIQUE INDEX  ===========
word_dict, tag_dict = create_corpus(X_train, y_train)

# =========== CREATE SEQUENCE LIST  ===========
# Next we will help ourselves with the nlp_hmms lecture and create a SequenceList object (without cython)
sequence_list = create_sequence_list(X_train, y_train, word_dict, tag_dict)

Creating Sequence List: 100%|██████████| 38366/38366 [02:20<00:00, 272.60it/s]


In [ ]:
# =========== MODELS - DEFAULT FEATURES ===========

# =========== GET FEATURES ===========
mapping_feature = IDFeatures(sequence_list)
mapping_feature.build_features()

# =========== MODEL PARAMETERS ===========
epochs = 15
structured_perceptron = StructuredPerceptron(word_dict, tag_dict, mapping_feature)
structured_perceptron.num_epochs = 5

# =========== MODEL TRAIN ===========
%time structured_perceptron.fit(mapping_feature.dataset, epochs)

# =========== MODEL SAVE ===========
structured_perceptron.save_model("fitted_models/default_features_model_wo_cython")

In [8]:
# =========== MODELS - ADDED FEATURES ===========
import skseq.sequences.structured_perceptron as spc
from skseq.sequences import extended_feature

# =========== GET FEATURES ===========
extra_mapping_feature = extended_feature.ExtendedFeatures(sequence_list) 
extra_mapping_feature.build_features()

# =========== MODEL PARAMETERS ===========
epochs = 15
structured_perceptron_extra = spc.StructuredPerceptron(word_dict, tag_dict, extra_mapping_feature)
structured_perceptron_extra.num_epochs = 5
%time structured_perceptron_extra.fit(extra_mapping_feature.dataset, epochs)

# =========== MODEL SAVE ===========
structured_perceptron_extra.save_model("fitted_models/extra_features_model_wo_cython.pkl")